In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ceres_infer.session import workflow
from ceres_infer.models import model_infer_ens_custom

/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/tensorflow/python/framework

In [3]:
import logging
logging.basicConfig(level=logging.INFO)

In [4]:
params = {
    # directories
    'outdir_run': '../out/20.0909 Lx/L100only_reg_rf_boruta/', # output dir for the run
    'outdir_modtmp': '../out/20.0909 Lx/L100only_reg_rf_boruta/model_perf/', # intermediate files for each model
    'indir_dmdata_Q3': '../out/20.0817 proc_data/gene_effect/dm_data.pkl', # pickled preprocessed DepMap Q3 data
    'indir_dmdata_Q4': '../out/20.0817 proc_data/gene_effect/dm_data_Q4.pkl', # pickled preprocessed DepMap Q3 data
    'indir_genesets': '../data/gene_sets/',
    'indir_landmarks': '../out/19.1013 tight cluster/landmarks_n100_k100.csv', # csv file of landmarks [default: None]

    # notes
    'session_notes': 'L100 landmarks only; regression with random forest-boruta lite iteration',

    # data
    'opt_scale_data': False, # scale input data True/False
    'opt_scale_data_types': '\[(?:RNA-seq|CN)\]', # data source types to scale; in regexp
    'model_data_source': ['CERES_Lx'],
    'anlyz_set_topN': 10, # for analysis set how many of the top features to look at
    'perm_null': 1000, # number of samples to get build the null distribution, for corr
    'useGene_dependency': False, # whether to use CERES gene dependency (true) or gene effect (false)
    'scope': 'differential', # scope for which target genes to run on; list of gene names, or 'all', 'differential'

    # model
    'model_name': 'rf',
    'model_params': {'n_estimators':1000,'max_depth':15,'min_samples_leaf':5,'max_features':'log2'},
    'model_paramsgrid': {},
    'model_pipeline': model_infer_ens_custom,
    'pipeline_params': {'sf_iterThresholds': [], 'sf_topK': None},
    
    # pipline
    'parallelize': False, # parallelize workflow
    'processes': 1, # number of cpu processes to use
    
    # analysis
    'metric_eval': 'score_test',  # metric in model_results to evaluate, e.g. score_test, score_oob
    'thresholds': {'score_rd10': 0.1,  # score of reduced model - threshold for filtering
                   'recall_rd10': 0.95},  # recall of reduced model - threshold for filtering
    'min_gs_size': 4 # minimum gene set size, to be derived
}

In [11]:
wf = workflow(params)
pipeline = ['load_processed_data', 'infer']
wf.create_pipe(pipeline)
wf.run_pipe()

INFO:root:Loading preprocessed data...
INFO:root:Adding landmarks...
INFO:root:Running model building and inference...
100%|██████████| 535/535 [4:33:22<00:00, 30.66s/it]  


In [13]:
wf = workflow(params)
pipeline = ['load_processed_data', 'load_model_results', 'analyze', 'analyze_filtered', 'derive_genesets']
wf.create_pipe(pipeline)
wf.run_pipe()

/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
INFO:root:Loading preprocessed data...
INFO:root:Adding landmarks...
INFO:root:Loading model results...
INFO:root:Analyzing model results...
/Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/analyses.py:303: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  feat_summary = varExp_noNeg.groupby('target')['target', 'score_rd', 'score_full'].first()
/Users/boyangzhao/Dropbox/Industry/Quantalarity/client Penn/proj_ceres/github/cnp_dev/src/ceres_infer/analyses.py:36: MatplotlibDeprecationWarning: normalize=None does not normalize if the sum is less than 1 but this behavior is deprecated since 3.3 until two minor releases la